In [ ]:
#changed options in tkinter

In [1]:
import tkinter as tk
from tkinter import *
import cv2
import csv
import os
import numpy as np
from PIL import Image,ImageTk
import pandas as pd
import datetime
import time

In [2]:
#####Window is our Main frame of system
window = tk.Tk()
window.title("FSMS-Face Recognition Based Security Management System")
window.configure(borderwidth=0)

displaystring=StringVar()
status = Label(window, textvariable=displaystring,font=('times',12),fg='#444444')
status.grid(row=8,columnspan=8,sticky='W',pady=(10,0))

In [3]:
# manually fill log

def manually_fill():
    global sb
    sb = tk.Tk()
    sb.iconbitmap('AMS.ico')
    sb.title("Manually Fill Loggg")
    #sb.geometry('580x320')

    def err_screen_for_subject():
        def ec_delete():
            ec.destroy()
            
        global ec
        ec = tk.Tk()
        ec.geometry('300x100')
        ec.iconbitmap('AMS.ico')
        ec.title('Warning!!')
        Label(ec, text='Please enter a valid room number!', font=('times', 16)).pack()
        Button(ec, text='OK', command=ec_delete,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1,
               font=('times', 15 )).place(x=90, y=50)

    def fill_attendance():
        ts = time.time()
        Date = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d')
        timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
        Time = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
        Hour, Minute, Second = timeStamp.split(":")
        #Creatting csv of attendance

        #Create table for Attendance
        date_for_DB = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d')
        global subb
        subb=SUB_ENTRY.get()
        DB_table_name = str(subb + "_" + Date + "_Time_" + Hour + "_" + Minute + "_" + Second)

        import pymysql.connections              ###Connect to the database
        try:
            global cursor
            connection = pymysql.connect(host='localhost', user='root', password='', db='manually_fill_attendance')
            cursor = connection.cursor()
        except Exception as e:
            print(e)

        sql = "CREATE TABLE " + DB_table_name + """
                        (ID INT NOT NULL AUTO_INCREMENT,
                         ENROLLMENT varchar(100) NOT NULL,
                         NAME VARCHAR(50) NOT NULL,
                         DATE VARCHAR(20) NOT NULL,
                         TIME VARCHAR(20) NOT NULL,
                             PRIMARY KEY (ID)
                             );
                        """

        try:
            cursor.execute(sql)               ##for create a table
        except Exception as ex:
            print(ex)

            #### message box
        if subb == '':
            err_screen_for_subject()
        else:
            sb.destroy()
            
            MFW = tk.Tk()
            MFW.iconbitmap('AMS.ico')
            MFW.title("Manual log of "+ str(subb))
            #MFW.geometry('880x470')

            def del_errsc2():
                errsc2.destroy()

            def err_screen1():
                global errsc2
                errsc2 = tk.Tk()
                errsc2.geometry('330x100')
                errsc2.iconbitmap('AMS.ico')
                errsc2.title('Warning!!')
                Label(errsc2, text='Please enter Name & ID!!!', font=('times', 16)).pack()
                Button(errsc2, text='OK', command=del_errsc2, width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times', 15)).place(x=90, y=50)

            def testVal(inStr, acttyp):
                if acttyp == '1':  # insert
                    if not inStr.isdigit():
                        return False
                return True

            ENR = tk.Label(MFW, text="Employee ID :",  font=('times', 15))
            ENR.grid(row=0,column=0,sticky=('S','W'),padx=10,pady=(10,0))
            #ENR.place(x=30, y=100)

            STU_NAME = tk.Label(MFW, text="Employee name :",font=('times', 15))
            STU_NAME.grid(row=2,column=0,sticky=('S','W'),padx=10,pady=(10,0))
            #STU_NAME.place(x=30, y=200)

            global ENR_ENTRY
            ENR_ENTRY = tk.Entry(MFW,width=20,validate='key', bg="white", font=('times', 23))
            ENR_ENTRY['validatecommand'] = (ENR_ENTRY.register(testVal), '%P', '%d')
            ENR_ENTRY.grid(row=1,column=0,padx=(10,0),pady=(3,10))
            #ENR_ENTRY.place(x=290, y=105)

            def remove_enr():
                ENR_ENTRY.delete(first=0, last=22)

            STUDENT_ENTRY = tk.Entry(MFW, width=20, bg="white", font=('times', 23))
            STUDENT_ENTRY.grid(row=3,column=0,padx=(10,0),pady=(3,10))
            #place(x=290, y=205)

            def remove_student():
                STUDENT_ENTRY.delete(first=0, last=22)

            ####get important variable
            def enter_data_DB():
                ENROLLMENT = ENR_ENTRY.get()
                STUDENT = STUDENT_ENTRY.get()
                if ENROLLMENT=='':
                    err_screen1()      #messagebox
                elif STUDENT=='':
                    err_screen1()
                else:
                    time = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                    Hour, Minute, Second = time.split(":")
                    Insert_data = "INSERT INTO " + DB_table_name + " (ID,ENROLLMENT,NAME,DATE,TIME) VALUES (0, %s, %s, %s,%s)"
                    VALUES = (str(ENROLLMENT), str(STUDENT), str(Date), str(time))
                    try:
                        cursor.execute(Insert_data, VALUES)
                    except Exception as e:
                        print(e)
                    ENR_ENTRY.delete(first=0, last=22)
                    STUDENT_ENTRY.delete(first=0, last=22)

            def create_csv():
                import csv
            
                cursor.execute("select * from " + DB_table_name + ";")
                csv_name='E:/Attendace managemnt system/Attendance/Manually Attendance/'+DB_table_name+'.csv'
                with open(csv_name, "w") as csv_file:
                    csv_writer = csv.writer(csv_file)
                    csv_writer.writerow([i[0] for i in cursor.description])  # write headers
                    csv_writer.writerows(cursor)
                    O="CSV created Successfully"
                    Notifi.configure(text=O, bg="Green", fg="white", width=33, font=('times', 19))
                    Notifi.place(x=180, y=380)
                import csv
                
                
                import tkinter
                root = tkinter.Tk()
                root.title("log of " + subb)
                with open(csv_name, newline="") as file:
                    reader = csv.reader(file)
                    r = 0
                    for col in reader:
                        c = 0
                        for row in col:
                            # i've added some styling
                            label = tkinter.Label(root, width=13, height=1, fg="black", font=('times', 13, ' bold '),
                                                  bg="white", text=row, relief=tkinter.RIDGE)
                            label.grid(row=r, column=c)
                            c += 1
                        r += 1
                root.mainloop()

            Notifi = tk.Label(MFW, text="CSV created Successfully", font=('times', 19))


            
            ####### one clear button for both
            c1ear_enroll = tk.Button(MFW, text="clear", command=remove_enr,activebackground='#999999',bg='#b9b9b9',borderwidth=0,font=('times', 15, ' bold '))
            c1ear_enroll.grid(row=1,column=1,padx=(0,10),sticky='W',pady=(3,10))
            #c1ear_enroll.place(x=690, y=100)

            c1ear_student = tk.Button(MFW, text="clear", command=remove_student,activebackground='#999999',bg='#b9b9b9',borderwidth=0, font=('times', 15, ' bold '))
            c1ear_student.grid(row=3,column=1,padx=(0,10),sticky='W',pady=(3,10))
            #c1ear_student.place(x=690, y=200)
        ###

        
            DATA_SUB = tk.Button(MFW, text="Add more",command=enter_data_DB,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1, font=('times', 15))
            DATA_SUB.grid(row=4,column=0,padx=10,pady=10)
            #DATA_SUB.place(x=170, y=300)

            MAKE_CSV = tk.Button(MFW, text="Convert to CSV",command=create_csv,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times', 15))
            MAKE_CSV.grid(row=4,column=1,padx=10,pady=10)
            #MAKE_CSV.place(x=570, y=300)

            #def attf():
            #    import subprocess
            #    subprocess.Popen(r'explorer /select,"E:/Attendace managemnt system\Attendance\Manually Attendance\-------Check atttendance-------"')

            #attf = tk.Button(MFW,  text="Check Sheets",command=attf,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1 ,font=('times', 14, ' bold '))
            #attf.place(x=730, y=410)

            status = Label(MFW, textvariable=displaystring,font=('times',12),fg='#444444')
            status.grid(row=8,columnspan=8,sticky='W',pady=(10,0))
            
            MFW.mainloop()


    SUB = tk.Label(sb, text="Enter room number", font=('times', 15))
    SUB.grid(row=0,column=0,sticky=('S','W'),padx=10,pady=(10,0))
    #SUB.place(x=30, y=100)

    global SUB_ENTRY

    SUB_ENTRY = tk.Entry(sb, width=20, bg="white", fg="black", font=('times', 23, ' bold '))
    SUB_ENTRY.grid(row=1,column=0,padx=10,pady=(0,10))
    #SUB_ENTRY.place(x=250, y=105)

    fill_manual_attendance = tk.Button(sb, text="Fill log",command=fill_attendance, width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1, font=('times', 15))
    fill_manual_attendance.grid(row=2,column=0,sticky=('E'),padx=10,pady=5)
    #fill_manual_attendance.place(x=250, y=160)
    
    status = Label(sb, textvariable=displaystring,font=('times',12),fg='#444444')
    status.grid(row=8,columnspan=8,sticky='W',pady=(10,0))
    
    sb.mainloop()

In [4]:
##For clear textbox
def clearall():
    txt.delete(first=0, last=22)
    txt2.delete(first=0, last=22)
    displaystring.set("click on 'Take Images' to proceed")
    
def clear():
    txt.delete(first=0, last=22)
def clear1():
    txt2.delete(first=0, last=22)
def del_sc1():
    sc1.destroy()
def err_screen():
    global sc1
    sc1 = tk.Tk()
    sc1.geometry('300x100')
    sc1.iconbitmap('AMS.ico')
    sc1.title('Warning!!')
    #sc1.configure(background='snow')
    Label(sc1,text='ID & Name required!!!',font=('times', 16, ' bold ')).pack()
    Button(sc1,text='OK',command=del_sc1 ,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times', 15)).place(x=90,y= 50)

##Error screen2
def del_sc2():
    sc2.destroy()
def err_screen1():
    global sc2
    sc2 = tk.Tk()
    sc2.geometry('300x100')
    sc2.iconbitmap('AMS.ico')
    sc2.title('Warning!!')
    #sc2.configure(background='snow')
    Label(sc2,text='Please enter the room number!!!',font=('times', 16)).pack()
    Button(sc2,text='OK',command=del_sc2  ,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times', 15)).place(x=90,y= 50)


In [5]:
# take images

def take_img():
    l1 = txt.get()
    l2 = txt2.get()
    if l1 == '':
        err_screen()         #### add message boxes
    elif l2 == '':
        err_screen()
    else:
        try:
            cam = cv2.VideoCapture(0)
            detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
            Enrollment = txt.get()
            Name = txt2.get()
            sampleNum = 0
            while (True):
                ret, img = cam.read()
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = detector.detectMultiScale(gray, 1.3, 5)
                for (x, y, w, h) in faces:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    # incrementing sample number
                    sampleNum = sampleNum + 1
                    # saving the captured face in the dataset folder
                    cv2.imwrite("TrainingImage/ " + Name + "." + Enrollment + '.' + str(sampleNum) + ".jpg",
                                gray[y:y + h, x:x + w])
                    cv2.imshow('Frame', img)
                # wait for 100 miliseconds
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                # break if the sample number is morethan 100
                elif sampleNum > 70:
                    break
            cam.release()
            cv2.destroyAllWindows()
            ts = time.time()
            Date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
            Time = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
            row = [Enrollment, Name, Date, Time]
            with open('StudentDetails\StudentDetails.csv', 'a+') as csvFile:
                writer = csv.writer(csvFile, delimiter=',')
                writer.writerow(row)
                csvFile.close()
                
                #####statusbar
            res = "Images Saved for Employee with ID : " + Enrollment + " Name : " + Name
            Notification.configure(text=res, bg="SpringGreen3", width=50, font=('times', 18, 'bold'))
            Notification.place(x=250, y=400)
            
            
        except FileExistsError as F:
            ####message box
            f = 'Employee Data already exists'
            Notification.configure(text=f, bg="Red", width=21)
            Notification.place(x=450, y=400)

In [6]:
#automatic fill log

def subjectchoose():
    def Fillattendances():
        sub=tx.get()
        now = time.time()  ###For calculate seconds of video
        future = now + 20
        if time.time() < future:
            if sub == '':
                err_screen1()     ##add error message instead
            else:
                recognizer = cv2.face.LBPHFaceRecognizer_create()  # cv2.createLBPHFaceRecognizer()
                try:
                    recognizer.read("TrainingImageLabel\Trainner.yml")
                except:
                    e = 'Model not found,Please train model'   #add error message box
                    Notifica.configure(text=e, bg="red", fg="black", width=33, font=('times', 15, 'bold'))
                    Notifica.place(x=20, y=250)

                harcascadePath = "haarcascade_frontalface_default.xml"
                faceCascade = cv2.CascadeClassifier(harcascadePath)
                df = pd.read_csv("StudentDetails\StudentDetails.csv")
                cam = cv2.VideoCapture(0)
                font = cv2.FONT_HERSHEY_SIMPLEX
                col_names = ['Enrollment', 'Name', 'Date', 'Time']
                attendance = pd.DataFrame(columns=col_names)
                while True:
                    ret, im = cam.read()
                    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                    faces = faceCascade.detectMultiScale(gray, 1.2, 5)
                    for (x, y, w, h) in faces:
                        global Id

                        Id, conf = recognizer.predict(gray[y:y + h, x:x + w])
                        if (conf <70):
                            print(conf)
                            global Subject
                            global aa
                            global date
                            global timeStamp
                            Subject = tx.get()
                            ts = time.time()
                            date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                            timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                            aa = df.loc[df['Enrollment'] == Id]['Name'].values
                            global tt
                            tt = str(Id) + "-" + aa
                            En = '15624031' + str(Id)
                            attendance.loc[len(attendance)] = [Id, aa, date, timeStamp]
                            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 260, 0), 7)
                            cv2.putText(im, str(tt), (x + h, y), font, 1, (255, 255, 0,), 4)

                        else:
                            Id = 'Unknown'
                            tt = str(Id)
                            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 25, 255), 7)
                            cv2.putText(im, str(tt), (x + h, y), font, 1, (0, 25, 255), 4)
                    if time.time() > future:
                        break

                    attendance = attendance.drop_duplicates(['Enrollment'], keep='first')
                    cv2.imshow('Filling attedance..', im)
                    key = cv2.waitKey(15) & 0xff
                    if key == 27:
                        break

                ts = time.time()
                date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                Hour, Minute, Second = timeStamp.split(":")
                fileName = "Attendance/" + Subject + "_" + date + "_" + Hour + "-" + Minute + "-" + Second + ".csv"
                attendance = attendance.drop_duplicates(['Enrollment'], keep='first')
                print(attendance)
                attendance.to_csv(fileName, index=False)

                ##Create table for Attendance
                date_for_DB = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d')
                DB_Table_name = str( Subject + "_" + date_for_DB + "_Time_" + Hour + "_" + Minute + "_" + Second)
                import pymysql.connections

                ###Connect to the database
                try:
                    global cursor
                    connection = pymysql.connect(host='localhost', user='root', password='', db='Face_reco_fill')
                    cursor = connection.cursor()
                except Exception as e:
                    print(e)

                sql = "CREATE TABLE " + DB_Table_name + """
                (ID INT NOT NULL AUTO_INCREMENT,
                 ENROLLMENT varchar(100) NOT NULL,
                 NAME VARCHAR(50) NOT NULL,
                 DATE VARCHAR(20) NOT NULL,
                 TIME VARCHAR(20) NOT NULL,
                     PRIMARY KEY (ID)
                     );
                """
                ####Now enter attendance in Database
                insert_data =  "INSERT INTO " + DB_Table_name + " (ID,ENROLLMENT,NAME,DATE,TIME) VALUES (0, %s, %s, %s,%s)"
                VALUES = (str(Id), str(aa), str(date), str(timeStamp))
                try:
                    cursor.execute(sql)  ##for create a table
                    cursor.execute(insert_data, VALUES)##For insert data into table
                except Exception as ex:
                    print(ex)  #

                    ####### create status bar  for This#####
                M = 'log filled Successfully'
                Notifica.configure(text=M, bg="Green", fg="white", width=33, font=('times', 15, 'bold'))
                Notifica.place(x=20, y=250)

                cam.release()
                cv2.destroyAllWindows()

                import csv
                import tkinter
                root = tkinter.Tk()
                root.title("Log of " + Subject)
                cs = 'E:/Attendace managemnt system/' + fileName
                with open(cs, newline="") as file:
                    reader = csv.reader(file)
                    r = 0

                    for col in reader:
                        c = 0
                        for row in col:
                            # i've added some styling
                            label = tkinter.Label(root, width=8, height=1, fg="black", font=('times', 15),
                                                  bg="white", text=row, relief=tkinter.RIDGE)
                            label.grid(row=r, column=c)
                            c += 1
                        r += 1
                root.mainloop()
                print(attendance)

    ###windo is frame for subject chooser
    windo = tk.Tk()
    windo.iconbitmap('AMS.ico')
    windo.title("Automatically fill log") 
    #windo.geometry('580x320')
    Notifica = tk.Label(windo, text="log filled Successfully",  font=('times', 15))

    #def Attf():
    #    import subprocess
    #    subprocess.Popen(r'explorer /select,"E:/Attendace managemnt system\Attendance\-------Check atttendance-------"')

    #attf = tk.Button(windo,  text="Check Sheets",command=Attf,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times', 14))
    #attf.place(x=430, y=255)

    sub = tk.Label(windo, text="Room Number",  font=('times', 15))
    sub.grid(row=0,column=0,sticky=('W','S'),padx=10,pady=(10,0))
    #sub.place(x=30, y=100)

    tx = tk.Entry(windo, width=20, font=('times', 23))
    tx.grid(row=1,column=0,padx=10,pady=(3,5))
    #tx.place(x=250, y=105)

    fill_a = tk.Button(windo, text="Register",command=Fillattendances ,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1, font=('times', 15))
    fill_a.grid(row=2,column=0,sticky='E',padx=10,pady=10)
    #fill_a.place(x=250, y=160)
    
    status = Label(windo, textvariable=displaystring,font=('times',12),fg='#444444')
    status.grid(row=8,columnspan=8,sticky='W',pady=(10,0))
    
    windo.mainloop()

def admin_panel():
    win = tk.Tk()
    win.iconbitmap('AMS.ico')
    win.title("Admin Login")
    #win.geometry('880x420')

    def log_in():
        username = un_entr.get()
        password = pw_entr.get()

        if username == 'k' :
            if password == 'k':
                win.destroy()
                
                import csv
                import tkinter
                root = tkinter.Tk()
                root.title("Employee Details")

                cs = 'E:/Attendace managemnt system/StudentDetails/StudentDetails.csv'
                with open(cs, newline="") as file:
                    reader = csv.reader(file)
                    r = 0
                    for col in reader:
                        c = 0
                        for row in col:
                            # i've added some styling
                            label = tkinter.Label(root, width=8, font=('times', 15), text=row, relief=tkinter.RIDGE)
                            label.grid(row=r, column=c)
                            c += 1
                        r += 1
                root.mainloop()
            else:
                
            #######message box
                valid = 'Incorrect ID or Password'
                Nt.configure(text=valid, bg="red", fg="black", width=38, font=('times', 19, 'bold'))
                Nt.place(x=120, y=350)
        else:
            valid ='Incorrect ID or Password'
            Nt.configure(text=valid, bg="red", fg="black", width=38, font=('times', 19, 'bold'))
            Nt.place(x=120, y=350)


    Nt = tk.Label(win, text="log filled Successfully",  font=('times', 19))
    # Nt.place(x=120, y=350)

    un = tk.Label(win, text="Enter username :",font=('times', 15))
    un.grid(row=0,column=0,sticky=('S','W'),padx=10,pady=(10,0))
    #un.place(x=30, y=50)

    pw = tk.Label(win, text="Enter password :", font=('times', 15))
    pw.grid(row=2,column=0,sticky=('S','W'),padx=10,pady=(10,0))
    #pw.place(x=30, y=150)

    def c00():
        un_entr.delete(first=0, last=22)

    un_entr = tk.Entry(win, width=20, font=('times', 23))
    un_entr.grid(row=1,column=0,padx=(10,0),pady=(3,10))
    #un_entr.place(x=290, y=55)

    def c11():
        pw_entr.delete(first=0, last=22)

    pw_entr = tk.Entry(win, width=20,show="*",  font=('times', 23))
    pw_entr.grid(row=3,column=0,padx=(10,0),pady=(3,10))
    #pw_entr.place(x=290, y=155)

    #### make one clear button for both
    
    c0 = tk.Button(win, text="Clear", command=c00,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=0, font=('times', 15))
    c0.grid(row=1,column=1,sticky=('W'),pady=(3,10),padx=(0,10))
    #c0.place(x=690, y=55)

    c1 = tk.Button(win, text="Clear", command=c11,width=13 ,activebackground='#999999',bg='#b9b9b9',borderwidth=0, font=('times', 15))
    c1.grid(row=3,column=1,sticky=('W'),pady=(3,10),padx=(0,10))
    #c1.place(x=690, y=155)
    
    ####

    Login = tk.Button(win, text="Login", command=log_in,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1, font=('times', 15))
    Login.grid(row=4,column=0,sticky='E',padx=10,pady=10)
    #Login.place(x=290, y=250)
    
    status = Label(win, textvariable=displaystring,font=('times',12),fg='#444444')
    status.grid(row=8,columnspan=8,sticky='W',pady=(10,0))
    
    win.mainloop()


In [7]:
#######    For train the model     ########

def trainimg():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    global detector
    detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    try:
        global faces,Id
        faces, Id = getImagesAndLabels("TrainingImage")
    except Exception as e:
        ## error message#############
        l='please make "TrainingImage" folder & put Images'
        #
        Notification.configure(text=l, bg="white", width=50, font=('times', 18, 'bold'))
        Notification.place(x=350, y=400)

    recognizer.train(faces, np.array(Id))
    try:
        recognizer.save("TrainingImageLabel\Trainner.yml")
    except Exception as e:
        q='Please make "TrainingImageLabel" folder'
        Notification.configure(text=q, bg="SpringGreen3", width=50, font=('times', 18, 'bold'))
        Notification.place(x=350, y=400)

    res = "Model Trained"  # +",".join(str(f) for f in Id)
    Notification.configure(text=res, bg="SpringGreen3", width=50, font=('times', 18, 'bold'))
    Notification.grid(row=8,columnspan=8,sticky='W',pady=(5,0))
    #Notification.place(x=250, y=400)

def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    # create empth face list
    faceSamples = []
    # create empty ID list
    Ids = []
    # now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:
        # loading the image and converting it to gray scale
        pilImage = Image.open(imagePath).convert('L')
        # Now we are converting the PIL image into numpy array
        imageNp = np.array(pilImage, 'uint8')
        # getting the Id from the image

        Id = int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces = detector.detectMultiScale(imageNp)
        # If a face is there then append that in the list as well as Id of it
        for (x, y, w, h) in faces:
            faceSamples.append(imageNp[y:y + h, x:x + w])
            Ids.append(Id)
    return faceSamples, Ids

#############################

window.grid_rowconfigure(0, weight=1)
window.grid_columnconfigure(0, weight=1)
window.resizable(False, False)
window.iconbitmap('AMS.ico')

Notification = tk.Label(window, text="All things good", bg="Green", fg="white", width=15,
                      height=3, font=('times', 17, 'bold'))

lbl = tk.Label(window, text="Employee ID", font=('times', 15))
lbl.grid(row=0, column=0,sticky='W')

def testVal(inStr, acttyp):
    if acttyp == '1': #insert
        if not inStr.isdigit():
            return False
    return True

txt = tk.Entry(window, validate="key", width=20, bg="white", font=('times', 20))
txt['validatecommand'] = (txt.register(testVal),'%P','%d')
txt.grid(row=0,column=1)

lbl2 = tk.Label(window, text="Employee Name", font=('times', 15))
lbl2.grid(row=1,column=0,sticky='W')

txt2 = tk.Entry(window, width=20, bg="white",font=('times', 20))
txt2.grid(row=1,column=1)

EnterBtn = tk.Button(window, text="Register",width=13, command =clearall,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times',15))
EnterBtn.grid(row=2,column=1,sticky='E',pady=5)

clearButton = tk.Button(window,text="clear",command=clear,activebackground='#999999',bg='#b9b9b9',borderwidth=0,font=('times',15))
clearButton.grid(row=0,column=2,sticky='W',pady=5)

clearButton1 = tk.Button(window,text="clear",command=clear1,activebackground='#999999',bg='#b9b9b9',borderwidth=0,font=('times',15))
clearButton1.grid(row=1,column=2,sticky='W',pady=5)

#maybe after pressing enter, open new window with cam on and take images button

AP = tk.Button(window, text="Check log",command=admin_panel,width=13,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times',15 ))
AP.grid(row=3,column=3,pady=10,padx=(45,10))

takeImg = tk.Button(window, text="Take Images",width=13,command=take_img,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times', 15))
takeImg.grid(row=4,column=0,pady=5,padx=10)

trainImg = tk.Button(window, text="Train Images",width=13,command=trainimg,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=( 'times',15))
trainImg.grid(row=4,column=1,pady=5,padx=10)

FA = tk.Button(window, text="Automatic log",width=13,command=subjectchoose,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times',15 ))
FA.grid(row=4,column=2,pady=5,padx=10)

quitWindow = tk.Button(window, text="Manual log", command=manually_fill, activebackground='#999999',width=13,bg='#b9b9b9',borderwidth=1,font=('times',15))
quitWindow.grid(row=4,column=3,pady=5,padx=(45,10))

window.mainloop()

65.53288097409032
55.15282069694625
64.57952309197155
55.64908880097377
65.12314993066397
55.9584138675825
64.51630423310377
63.567228678993025
63.69795182456283
63.191687839984056
64.33552233075427
64.6873744528932
63.87279867063683
63.83912425680421
63.8464194529073
64.67100124118421
64.43146889325261
62.78394153935966
63.75244023559271
65.08088916322733
63.904236023996454
64.29499840714297
63.54948979262034
64.81343556594855
62.55866329687534
64.28115470585895
64.19096058881635
55.41709917027058
64.27476716007912
63.67070314225293
55.97253647431971
65.1237607495022
63.630231780917846
63.32662876389475
65.14377526810362
63.80676095115403
53.90763436112952
53.90763436112952
63.717251588941664
63.06232851703914
63.06232851703914
65.176490077601
56.98592820373776
63.5230433595791
63.5230433595791
65.03256132943132
64.07527936408607
65.01645960920882
66.14760955445688
66.14760955445688
65.63395452159791
65.61435395490382
66.48633404980134
66.48633404980134
58.38113457672662
59.3392802588